In [1]:
import torch
from monai.networks.nets import UNet
from monai.inferers import sliding_window_inference

### Load test tomograms

In [2]:
import numpy as np
from tqdm import tqdm
import copick

copick_config_path = "configs/test_config.json" 
root = copick.from_file(copick_config_path)

In [3]:
test_dataset = []
for run in tqdm(root.runs):
    tomo = run.get_voxel_spacing(10).get_tomogram('denoised').numpy()
    test_dataset.append({'image': tomo})


100%|██████████| 3/3 [00:00<00:00,  5.02it/s]


In [4]:
len(test_dataset)

3

### Create dataloader for the test dataset

In [5]:
from monai.data import DataLoader, CacheDataset
from monai.transforms import (
    Compose, 
    NormalizeIntensityd,
    EnsureChannelFirstd, 
    Activationsd,
    AsDiscreted
)

# define pre transforms
pre_transforms = Compose(
    [
        EnsureChannelFirstd(keys=["image"], channel_dim="no_channel"),
        NormalizeIntensityd(keys=["image"]),
    ]
)


test_ds = CacheDataset(data=test_dataset, transform=pre_transforms)
test_loader = DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=4, pin_memory=torch.cuda.is_available())

Loading dataset: 100%|██████████| 3/3 [00:00<00:00, 10.60it/s]


### Load model and weights

In [6]:
from models import CryoETUNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = CryoETUNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=len(root.pickable_objects)+1,
    channels=(48, 64, 80, 80),
    strides=(2, 2, 1),
    num_res_units=1,
).to(device)

model.load_state_dict(torch.load("models/baseline/best_metric_model.pth", weights_only=True))
model.eval()

cuda


CryoETUNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv3d(1, 48, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): InstanceNorm3d(48, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv3d(1, 48, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Sequential(
            (unit0): Convolution(
              (conv): Conv3d(48, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
              (adn): ADN(
                (N): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
                (D): Dropout(p=0.0, inplace=False)
                (A): PReLU(

### Inference

In [7]:
def inference(model, input):
    def _compute(input):
        return sliding_window_inference(
            inputs=input,
            roi_size=(96, 96, 96),
            sw_batch_size=4,  # one window is proecessed at a time
            predictor=model,
            overlap=0.5,
        )

    with torch.cuda.amp.autocast():
        return _compute(input)

In [8]:
from monai.data import decollate_batch
from tqdm import tqdm

post_transforms = Compose([
    Activationsd(keys="pred", softmax=True),
    AsDiscreted(keys="pred", argmax=True)
])

predictions = []
with torch.no_grad():
    for data in tqdm(test_loader):
        tomogram = data['image'].to(device)  # only support batch=1 and channel first
        data["pred"] = inference(model, tomogram)
        data = [post_transforms(i) for i in decollate_batch(data)]
        for b in data:
            predictions.append(b['pred'].squeeze(0).numpy(force=True))

  0%|          | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_18682/339001617.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 1/1 [00:06<00:00,  6.64s/it]


In [9]:
print(np.unique(predictions[0]))

[0. 1. 3. 4. 5. 6.]


In [10]:
import copick_utils
from copick_utils.segmentation.picks_from_segmentation import picks_from_segmentation


particles = dict()
for po in root.config.pickable_objects:
    particles[po.name] = po.label

    
maxima_filter_size = 10
min_particle_size = 0
max_particle_size = 10
new_session_id = "1"
new_user_id = "paintedFromInferencePicks"
for prediction, run in tqdm(zip(predictions, root.runs)):
    for po in particles.keys():
        if po != "membrane":
            class_label = particles[po]
            picks_from_segmentation(prediction, class_label, maxima_filter_size, min_particle_size, max_particle_size, new_session_id, new_user_id, po, run, voxel_spacing=10)

0it [00:00, ?it/s]/home/ngo.trun/miniconda3/envs/cryoet/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `CopickPoint` but got `dict` with value `{'x': 4140.0, 'y': 290.0, 'z': 10.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 1200.0, 'y': 2.0, 'z': 125.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 2400.0, 'y': 1710.0, 'z': 215.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 2390.0, 'y': 1720.0, 'z': 220.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 6280.0, 'y': 2210.0, 'z': 460.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 6288.333333333334, 'y': 2210.0, 'z': 490.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` 

Centroids for label 1 saved successfully.


/home/ngo.trun/miniconda3/envs/cryoet/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `CopickPoint` but got `dict` with value `{'x': 5880.0, 'y': 4340.0, 'z': 30.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5901.666666666666, 'y': 4340.0, 'z': 30.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5911.0, 'y': 4334.0, 'z': 41.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4160.0, 'y': 130.0, 'z': 50.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 1570.0, 'y': 880.0, 'z': 80.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 1580.0, 'y': 920.0, 'z': 90.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 2068

Centroids for label 3 saved successfully.


/home/ngo.trun/miniconda3/envs/cryoet/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `CopickPoint` but got `dict` with value `{'x': 2940.0, 'y': 470.0, 'z': 130.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 6120.0, 'y': 5112.0, 'z': 128.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 2930.0, 'y': 460.0, 'z': 140.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 6118.0, 'y': 5100.0, 'z': 138.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 6142.5, 'y': 5087.5, 'z': 147.5}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5187.777777777778, ...'z': 178.88888888888889}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `

...

Centroids for label 4 saved successfully.


/home/ngo.trun/miniconda3/envs/cryoet/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `CopickPoint` but got `dict` with value `{'x': 4260.0, 'y': 500.0, 'z': 30.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5330.0, 'y': 2010.0, 'z': 30.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5362.0, 'y': 2032.0, 'z': 40.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5352.0, 'y': 2020.0, 'z': 62.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5340.0, 'y': 2010.0, 'z': 70.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4590.0, 'y': 2270.0, 'z': 190.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4012.0, 'y':

Centroids for label 5 saved successfully.


/home/ngo.trun/miniconda3/envs/cryoet/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `CopickPoint` but got `dict` with value `{'x': 2260.0, 'y': 2380.0, 'z': 260.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 1060.0, 'y': 1860.0, 'z': 360.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 3370.0, 'y': 2820.0, 'z': 450.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4860.0, 'y': 460.0, 'z': 630.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4850.0, 'y': 450.0, 'z': 640.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4040.0, 'y': 6288.333333333334, 'z': 700.0}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x

Centroids for label 6 saved successfully.
No segmentation with label 9 found.


In [11]:
particles

{'apo-ferritin': 1,
 'beta-galactosidase': 3,
 'ribosome': 4,
 'thyroglobulin': 5,
 'virus-like-particle': 6,
 'membrane': 8,
 'background': 9}

In [21]:
import pandas as pd

inference_points = []
idx = -1

for run in tqdm(root.runs):
    print("------------------------------------")
    print("Experiment name: ", run.name)
    for pname in particles:
        pick = run.get_picks(object_name=pname, user_id=new_user_id)
        if pick:
            for p in pick[0].points:
                idx += 1
                inference_points.append({
                    'id': idx,
                    'experiment': run.name,
                    'particle_type': pname,
                    'x': p['x'],
                    'y': p['y'],
                    'z': p['z']
                })


df = pd.DataFrame(inference_points)
df.to_csv("submission.csv", index=False)
    

100%|██████████| 3/3 [00:00<00:00, 5899.16it/s]

------------------------------------
Experiment name:  TS_5_4
------------------------------------
Experiment name:  TS_69_2
------------------------------------
Experiment name:  TS_6_4
